Since what I'm most interested in is which factors are most associated with whether an issue is unresolved, random forests seems like the most appropriate model choice, since I can look at the variable importance plot.

Logistic regression is a close second; random forests usually have better performance.

Random forests can also do a better job at getting signal out of raw lat and long. For logistic regression, there would need to be a linear relationship between lat and long and the log-odds of the issue being unresolved, which is a taller order.

In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle('../data/data_w_transformed_census.pkl')
df.shape

(905205, 48)

In [3]:
df.head(1).T

,0
CASE_ENQUIRY_ID,101000958209
OPEN_DT,2013-11-01 09:27:19
TARGET_DT,2013-11-15 09:27:19
CLOSED_DT,2013-11-27 10:15:45
OnTime_Status,OVERDUE
CASE_STATUS,Closed
CLOSURE_REASON,Case Closed Case Resolved
CASE_TITLE,Sign Repair
SUBJECT,Transportation - Traffic Division
REASON,Signs & Signals


## Preprocessing

In [ ]:
df['is_issue']